In [19]:
import sys
import os
from pathlib import Path
from collections import OrderedDict
from pprint import pprint, pformat

In [20]:
import cv2
import numpy as np
import pandas as pd
import matplotlib
from IPython.display import display


In [72]:
boxname = "Box1"
print("\n".join(f"dm{num:02},{boxname}" for num in range(20)))
boxname = "Box2"
print("\n".join(f"dm{num:02},{boxname}" for num in range(20, 35)))


dm00,Box1
dm01,Box1
dm02,Box1
dm03,Box1
dm04,Box1
dm05,Box1
dm06,Box1
dm07,Box1
dm08,Box1
dm09,Box1
dm10,Box1
dm11,Box1
dm12,Box1
dm13,Box1
dm14,Box1
dm15,Box1
dm16,Box1
dm17,Box1
dm18,Box1
dm19,Box1
dm20,Box2
dm21,Box2
dm22,Box2
dm23,Box2
dm24,Box2
dm25,Box2
dm26,Box2
dm27,Box2
dm28,Box2
dm29,Box2
dm30,Box2
dm31,Box2
dm32,Box2
dm33,Box2
dm34,Box2


In [1]:
import string
for r in "ABCDEFGH":
    for c in range(1, 10):
        print(f"{r}{c:02}")

A01
A02
A03
A04
A05
A06
A07
A08
A09
B01
B02
B03
B04
B05
B06
B07
B08
B09
C01
C02
C03
C04
C05
C06
C07
C08
C09
D01
D02
D03
D04
D05
D06
D07
D08
D09
E01
E02
E03
E04
E05
E06
E07
E08
E09
F01
F02
F03
F04
F05
F06
F07
F08
F09
G01
G02
G03
G04
G05
G06
G07
G08
G09
H01
H02
H03
H04
H05
H06
H07
H08
H09


In [73]:
sys.path.insert(0, os.path.abspath('..'))
import zepto_lims

example_data_dir = Path(zepto_lims.__file__).parent / "examples" / "example_data"
example_data_dir

WindowsPath('C:/Users/au206270/Dev/Zepto-LIMS-Inventory-tube-tracker/zepto_lims/examples/example_data')

In [74]:
tubes_df = pd.read_csv(example_data_dir / 'Default_tubes.csv')
tubes_df.head()

,barcode,boxname
0,dm00,Box1
1,dm01,Box1
2,dm02,Box1
3,dm03,Box1
4,dm04,Box1


In [75]:
grouped = tubes_df.groupby('boxname')

In [76]:
type(grouped.Barcode)

AttributeError: 'DataFrameGroupBy' object has no attribute 'Barcode'

In [77]:
box1_barcodes = grouped.get_group('Box1')['barcode']
box1_barcodes
box1_barcodeset = set(box1_barcodes)
box1_barcodeset
print(",".join(box1_barcodeset))

dm06,dm05,dm02,dm11,dm19,dm08,dm18,dm10,dm15,dm04,dm12,dm17,dm07,dm16,dm00,dm01,dm14,dm13,dm09,dm03


In [78]:
for groupname, group_df in grouped:
    print(groupname)
    print(group_df['barcode'].values)
    print("\n")

Box1
['dm00' 'dm01' 'dm02' 'dm03' 'dm04' 'dm05' 'dm06' 'dm07' 'dm08' 'dm09'
 'dm10' 'dm11' 'dm12' 'dm13' 'dm14' 'dm15' 'dm16' 'dm17' 'dm18' 'dm19']


Box2
['dm20' 'dm21' 'dm22' 'dm23' 'dm24' 'dm25' 'dm26' 'dm27' 'dm28' 'dm29'
 'dm30' 'dm31' 'dm32' 'dm33' 'dm34']




In [79]:
barcodes_set = set("dm06,dm05,dm02,dm11,dm19,dm08,dm18,dm40,dm41,dm42".split(","))
barcodes_set

{'dm02',
 'dm05',
 'dm06',
 'dm08',
 'dm11',
 'dm18',
 'dm19',
 'dm40',
 'dm41',
 'dm42'}

In [80]:
barcodeset - box1_barcodeset  # New

{'dm40', 'dm41', 'dm42'}

In [93]:
tubes_grouped_by_box = tubes_df.groupby('boxname')
boxes_barcodesets = OrderedDict(
    {boxname: set(group_df['barcode'].values)
     for boxname, group_df in tubes_df.groupby('boxname')}
)
# common, added, missing
boxes_diff = {boxname: (
        box_barcodes & barcodes_set,  # intersection
        barcodes_set - box_barcodes,  # added
        box_barcodes - barcodes_set,  # removed
    )
    for boxname, box_barcodes in boxes_barcodesets.items()
}
print("scanned_barcodeset: ", sorted(barcodes_set))
print("Box1: ", sorted(boxes_barcodesets['Box1']))
print("Box2: ", sorted(boxes_barcodesets['Box2']))
print("\n\n".join(
    f"{boxname}\n Common: {sorted(common)}\n Added: {sorted(added)}\n Removed: {sorted(removed)}"
    for boxname, (common, added, removed) in boxes_diff.items()
))

box_similarity_scores = sorted([
    (len(common) - 0.2*len(added) - 0.1*len(removed), boxname) 
    for boxname, (common, added, removed) in boxes_diff.items()
], reverse=True)
box_similarity_scores

scanned_barcodeset:  ['dm02', 'dm05', 'dm06', 'dm08', 'dm11', 'dm18', 'dm19', 'dm40', 'dm41', 'dm42']
Box1:  ['dm00', 'dm01', 'dm02', 'dm03', 'dm04', 'dm05', 'dm06', 'dm07', 'dm08', 'dm09', 'dm10', 'dm11', 'dm12', 'dm13', 'dm14', 'dm15', 'dm16', 'dm17', 'dm18', 'dm19']
Box2:  ['dm20', 'dm21', 'dm22', 'dm23', 'dm24', 'dm25', 'dm26', 'dm27', 'dm28', 'dm29', 'dm30', 'dm31', 'dm32', 'dm33', 'dm34']
Box1
 Common: ['dm02', 'dm05', 'dm06', 'dm08', 'dm11', 'dm18', 'dm19']
 Added: ['dm40', 'dm41', 'dm42']
 Removed: ['dm00', 'dm01', 'dm03', 'dm04', 'dm07', 'dm09', 'dm10', 'dm12', 'dm13', 'dm14', 'dm15', 'dm16', 'dm17']

Box2
 Common: []
 Added: ['dm02', 'dm05', 'dm06', 'dm08', 'dm11', 'dm18', 'dm19', 'dm40', 'dm41', 'dm42']
 Removed: ['dm20', 'dm21', 'dm22', 'dm23', 'dm24', 'dm25', 'dm26', 'dm27', 'dm28', 'dm29', 'dm30', 'dm31', 'dm32', 'dm33', 'dm34']


[(5.1000000000000005, 'Box1'), (-3.5, 'Box2')]

In [95]:
box_similarity_scores = sorted([
    (len(common) - 0.2*len(added) - 0.1*len(removed), boxname)
    for boxname, (common, added, removed) in boxes_diff.items()
], reverse=True)
pprint(box_similarity_scores)


[(5.1000000000000005, 'Box1'), (-3.5, 'Box2')]


In [106]:
boxes_diff_count = OrderedDict(
    (boxname, (len(common), len(added), len(removed), len(common) - 0.2*len(added) - 0.1*len(removed)))
    for boxname, (common, added, removed) in boxes_diff.items()
)
boxes_diff_count_df = pd.DataFrame.from_records(
    list(boxes_diff_count.values()), 
    columns="common added removed similarity".split(),
    index=boxes_diff_count.keys()
)
boxes_diff_count_df.head()

,common,added,removed,similarity
Box1,7,3,13,5.1
Box2,0,10,15,-3.5


In [99]:
pd.DataFrame.from_records(boxes_diff)

,Box1,Box2
0,"{dm19, dm06, dm08, dm18, dm05, dm02, dm11}",{}
1,"{dm41, dm40, dm42}","{dm19, dm06, dm08, dm41, dm40, dm18, dm42, dm0..."
2,"{dm00, dm04, dm12, dm17, dm01, dm07, dm14, dm1...","{dm26, dm34, dm33, dm22, dm21, dm32, dm24, dm2..."


In [109]:
len(boxes_diff_count_df)

2

In [113]:
best_box = boxes_diff_count_df.index[0]
best_box

'Box1'

In [114]:
120/18

6.666666666666667

In [116]:
tubes_df.head()
tubes_df.where('boxname = Box1')

ValueError: Array conditional must be same shape as self

In [118]:
from datetime import datetime
filename = '{:%Y%m%d}'.format(datetime.now())
print(filename)

20190821


In [120]:
from datetime import datetime
from pathlib import Path
filename = Path('{:%Y%m%d}').format(datetime.now())
print(filename)

AttributeError: 'WindowsPath' object has no attribute 'format'

In [127]:
p = Path("~/non-existing-folder")
p = p.expanduser()
p.mkdir?

In [128]:
set(dict(a=1, b=2))

{'a', 'b'}

In [7]:
import re
pos_regex=r"(?P<row>[A-z])(?P<col>\d+)"
pos_regex = re.compile(pos_regex)
match = pos_regex.match("A01")
match.groupdict()
match.groups()

('A', '01')

In [11]:
a, b = 1, 2
a, b += 1, 1

a, b

SyntaxError: illegal expression for augmented assignment (<ipython-input-11-762c5aec569a>, line 2)

In [22]:
ks, vs = zip(*dict(a=1, b=2).items())
print(vs)
print(np.array(vs))
print(np.array(((1, 2), (3, 4))))

(1, 2)
[1 2]
[[1 2]
 [3 4]]


In [25]:
a = np.array(vs)
type(a)
isinstance(a, np.ndarray)

True

In [40]:
a = np.array(((1, 2), (3, 4), (5, 6)))
np.mean?
np.mean(a, axis=0)

array([3., 4.])

In [41]:
np.linalg.norm(a, axis=1)

array([2.23606798, 5.        , 7.81024968])

In [45]:
asq = a*a
print(a)
print(asq)
np.sqrt(asq[:, 0] + asq[:, 1])

[[1 2]
 [3 4]
 [5 6]]
[[ 1  4]
 [ 9 16]
 [25 36]]


array([2.23606798, 5.        , 7.81024968])

In [46]:
sorted([1, 2, 3])

[1, 2, 3]